# 구글 이미지 스크래핑

* 스크롤을 내려야만 다음 것을 선택
* 클릭후 오른쪽 창에 나온것이 원본 이미지

## Selenium 및 웹 드라이버 설치

In [ ]:
# # coloab
# !pip install Selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

# import sys
# sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

# from selenium import webdriver
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

# wd = webdriver.Chrome('chromedirver', options=chrome_options)

## 라이브러리 import

In [ ]:
import os
import time
import socket # 소켓 에러 방지

from selenium import webdriver
from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError # 각종 Error 대비
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image

## `scroll_down()` : 스크롤을 밑으로 내리는 함수

![image.png](https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/db18ea1b27fb47efa72eea8cc6971d80/f7f810b9-44f5-460e-a06f-4a9030a87994.png)

Xpath:

```
//*[@id="islmp"]/div/div/div/div/div[2]/div[2]/input
```

In [ ]:
def scroll_down(t=1):
    scroll_count = 0 
    print("[scroll_down(): 스크롤 다운 시작]")
    
    last_height = wd.execute_script('return document.body.scrollHeight')
    after_click = False
    while True:
        print(f"[스크롤 다운: {scroll_count}]")
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 스크롤을 내리는 명령어
        scroll_count += 1
        time.sleep(t)
        
        new_height = wd.execute_script('return document.body.scrollHeight')
        
        if last_height == new_height: # 내려 갈때가 없다면
            if after_click is True: # 결과 더보기 버튼이 있으면
                break
            
            else:
                try:
                    more_button = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[2]/div[2]/input')
                    if more_button.is_displayed():
                        more_button.click()
                        after_click = True
                except NoSuchElementException as e:
                    print(type(e))
                    break
                    
        last_height = new_height
    

    


## `click_and_save()` : 썸네일 이미지 선택 후 원본 이미지 저장


![img.save](https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/db18ea1b27fb47efa72eea8cc6971d80/785eb566-c6e0-4e72-b07a-2f1438f03ecd.png)

```
//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img

```

보통 png 또는 jpg

//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img

#Sva75c > div > div > div.pxAole > div.tvh9oe.BIB1wf > c-wiz > div > div.OUZ5W > div.zjoqD > div.qdnLaf.isv-id > div.v4dQwb > a > img

//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img

In [ ]:
'dataddfd'.startswith('data')

In [ ]:
def click_and_save(dir_name, index, img, img_list_length):
    global scraped_count
    
    try:
        img.click()
        time.sleep(0.1)
        cnt = 0 
        src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img').get_attribute('src')
        while src.startswith('data'):
            time.sleep(0.1)
            src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img').get_attribute('src')
            cnt +=1
            if cnt == 3:
                break
        
        if src.startswith('data'):
            print('data.img')
        else:
            print(src)
        
            if src.split('.')[-1] == 'png':
                urlretrieve(src, dir_name + '/' + str(scraped_count + 1 ) + '.png')
                print(f"{index+1}/{img_list_length} PNG 이미지 저장 완료")
            else:
                urlretrieve(src, dir_name + '/' + str(scraped_count + 1 ) + '.jpg')
                print(f"{index+1}/{img_list_length} JPG 이미지 저장 완료")

            scraped_count += 1
    
    except HTTPError as e:
        print(e)
        pass
    except ElementClickInterceptedException as e:
        print('Click intercepted  Error Ocurred')
        pass

## `scraping()` : 구글 이미지 스크래핑 함수 정의

![imag.xpath](https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/db18ea1b27fb47efa72eea8cc6971d80/acb22326-c460-4899-92fe-d38a03777450.png)

Xpath:

```
//*[@id="islrg"]/div[1]

```


![css.selector](https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/db18ea1b27fb47efa72eea8cc6971d80/e670a5dd-b8a7-4b7e-9dce-0208583a1602.png)

class="rg_i Q4LuWd"

* . 을 찍고 rg_i Q4LuWd 해주면 css_selector 에 넣으면 된다. 


----

`img_list = wd.find_elements_by_css_selector('.rg_i Q4LuWd')` 에서 결과 값을 못 가져 온다. 다른 것을 시도 해본다. 

`#islrg > div.islrc > div:nth-child(8) > a.wXeWr.islib.nfEiy > div.bRMDJf.islir > img` selector 에서

In [ ]:
def scraping(dir_name, query):
    global scraped_count
    
    url = f'https://www.google.co.kr/search?q={query}&tbm=isch&hl=en&tbs=isz:l&authuser=0&sa=X&ved=0CAIQpwVqFwoTCNDqnPW2vvECFQAAAAAdAAAAABAC&biw=826&bih=959'
    wd.get(url)
    wd.maximize_window() # 창을 가장 크게 해야 이미지 노출 수가 증가 함으로
    print(url)
    scroll_down()
    
    div = wd.find_element_by_xpath('//*[@id="islrg"]/div[1]')
    img_list = div.find_elements_by_css_selector('div.bRMDJf.islir > img')
#     print(img_list)
    print('ok')
    wd.execute_script("window.scrollTo(0, 1)") # 초반부에 자꾸 Element ClickInterception Error 발생 방지용
    time.sleep(5) # 맨 위로 올라가서 사용하기
    
    for index, img in enumerate(img_list):
        
        if index%10 ==1:
            time.sleep(3)
            
        print(index)
        try:
            click_and_save(dir_name, index, img, len(img_list))
        
        except ElementClickInterceptedException as e:
            print(e)
            wd.execute_script("window.scrollTo(0, window.scrollY + 100)")
            time.sleep(1)
            click_and_save(dir_name, index, img, len(img_list))
        
        except NoSuchElementException as e:
            print(e)
            wd.execute_script("window.scrollTo(0, window.scrollY + 100)")
            time.sleep(1)
            click_and_save(dir_name, index, img, len(img_list))            

        except ConnectionResetError as e:
            print(e)
            pass
            
        except URLError as e: 
            print(e)
            pass
        
        except socket.timeout as e: 
            print(e)
            pass
        
        except socket.gaierror as e: 
            print(e)
            pass
        
        except ElementNotInteractableException as e:
            print(e)
            break
    try: 
        print("[스크래핑 종료 (성공률: %.2f%%)]"%(scraped_count / len(img_list) * 100.0 ))
    
    except ZeroDivisionError as e: 
        print(e)
        
    wd.quit()
        

## `filter_and_remove()` : 일정 해상도 이하이거나 손상된 이미지 제거

In [ ]:
def filter_and_remove(dir_name, query, filter_size):
    filtered_count = 0 
    for index, file_name in enumerate(os.listdir(dir_name)):
        try:
            
            file_path = os.path.join(dir_name, file_name)
            img = Image.open(file_path)
            if img.width < filter_size and img.height < filter_size :
                img.close()
                os.remove(file_path)
                print(f'{index} 이미지 제거')
                filtered_count += 1
        except OSError as e: 
            print(e)
            os.remove(file_path)
            print(f'{index} 이미지 제거')
            filtered_count += 1
    else:
        print(f'[이미지 제거 개수: {filtered_count}/{scraped_count}]')
    



## 스크래핑 실행

In [ ]:
# os.removedirs('sea/')
socket.setdefaulttimeout(30) # 30초 이상 지체되면 끊어줘~

wd = webdriver.Chrome('chromedriver')
scraped_count = 0 

path = "./"
# query = input("검색어 입력:")
query = 'sea'

dir_name = path + query
os.makedirs(dir_name)
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query)

filter_size = 400
filter_and_remove(dir_name, query, filter_size)


문제점 발견 

* 원래의 의도는 고용량의 사진을 가져 오기 위한 셋팅이었는데 실제로는 작은 파일을 그대로 가지고 오고 있다. 

    - 해결 방법 time.sleep()의 부재로 생긴 일 : 클릭후 시간을 1.5초 정도 필요
    
* 처음 부분을 저용량으로 받아오는 문제 생김

    - 